# Build de Vector DB (Chroma) — Colab + local

> Objetivo: construir un artefacto Chroma persistente en disco para búsqueda semántica.

**Diseño:** este notebook se puede correr tanto en **Google Colab** (para datasets grandes / GPU) como en **local**.

**Output (en este repo):**
- `backend/agent/vector_db/` (persist_directory)
- `backend/agent/vector_db/manifest.json`
- (opcional) `backend/agent/vector_db.zip`


In [ ]:
# 0) Environment check (Colab/local)
import sys, platform
print('Python:', sys.version)
print('Platform:', platform.platform())

# Best-effort GPU check (sentence-transformers usa torch por debajo)
try:
    import torch

    print('torch:', torch.__version__)
    print('CUDA available:', torch.cuda.is_available())
    if torch.cuda.is_available():
        print('GPU:', torch.cuda.get_device_name(0))
except Exception as e:
    print('GPU check skipped:', repr(e))


## 1) Setup (opcional) — clonar repo (Colab)

Este notebook funciona en:
- **Local** (VS Code / Jupyter): abre el notebook dentro del repo.
- **Google Colab**: activa `DO_CLONE_REPO=True` y configura `REPO_URL`.


In [ ]:
# 2A) (Opcional) Clonar repo (pensado para Colab)
# Si ya estás en tu repo local, deja DO_CLONE_REPO=False.

import os

DO_CLONE_REPO = False
REPO_URL = 'https://github.com/<org-or-user>/<repo>.git'  # <-- cambia esto
REPO_DIR = 'Libreria-Aurora'  # carpeta destino (ajusta si necesitas)

if DO_CLONE_REPO:
    import subprocess
    from pathlib import Path

    if not Path(REPO_DIR).exists():
        print('Cloning:', REPO_URL)
        subprocess.run(['git', 'clone', REPO_URL, REPO_DIR], check=True)
    else:
        print('Repo dir already exists:', REPO_DIR)

    os.chdir(REPO_DIR)
    print('cwd ->', Path.cwd())
else:
    print('DO_CLONE_REPO=False (skip)')


## 2) Setup (opcional) — instalar dependencias

Si corres esto en Colab o en un entorno vacío, puedes activar `DO_INSTALL_DEPS=True` para instalar lo necesario desde `backend/requirements.txt`.


In [ ]:
# 2B) (Opcional) Instalar dependencias (Colab/local)
# - En VS Code normalmente ya tienes tu venv y esto no es necesario.
# - En Colab sí conviene correrlo una vez.

DO_INSTALL_DEPS = False  # <-- cambia a True si estás en Colab o quieres bootstrap automático

if DO_INSTALL_DEPS:
    import subprocess, sys

    def run(cmd) -> None:
        print('+', ' '.join(cmd))
        subprocess.run(cmd, check=True)

    run([sys.executable, '-m', 'pip', 'install', '--upgrade', 'pip'])
    run([sys.executable, '-m', 'pip', 'install', '-r', 'backend/requirements.txt'])
else:
    print('DO_INSTALL_DEPS=False (skip)')


In [ ]:
from __future__ import annotations

import hashlib
import json
import os
import shutil
from datetime import datetime, timezone
from pathlib import Path
from typing import Any

print('cwd:', Path.cwd())

## 3) Configuración de rutas y parámetros

Ajusta si tu `cwd` no es la raíz del repo. Por defecto intenta detectar la raíz buscando `docker-compose.yml` o la carpeta `backend/`.

Este notebook es genérico: puedes cambiar el dataset y el output directory sin tocar el resto.

In [ ]:
from pathlib import Path
import os


def find_repo_root(start: Path) -> Path:
    cur = start.resolve()
    for parent in [cur] + list(cur.parents):
        if (parent / 'docker-compose.yml').exists() and (parent / 'backend').exists():
            return parent
        if (parent / 'backend').exists() and (parent / 'docs').exists():
            return parent
    return cur


REPO_ROOT = find_repo_root(Path.cwd())
BACKEND_ROOT = REPO_ROOT / 'backend'

# Dataset
# - Default: fixture del repo
# - Genérico: puedes pasar un JSON alternativo vía env var
DEFAULT_DATASET_PATH = BACKEND_ROOT / 'apps' / 'libros' / 'fixtures' / 'libros_prueba.json'
DATASET_PATH = Path(os.getenv('VECTOR_DATASET_PATH', str(DEFAULT_DATASET_PATH)))

# Output (centralizado en backend/agent/)
VECTOR_DB_DIR = BACKEND_ROOT / 'agent' / 'vector_db'
MANIFEST_PATH = VECTOR_DB_DIR / 'manifest.json'
ZIP_PATH = BACKEND_ROOT / 'agent' / 'vector_db.zip'

# Puedes sobreescribir el nombre de colección desde env (alineado al plan)
VECTOR_COLLECTION = os.getenv('VECTOR_COLLECTION', 'book_catalog')

# Embeddings (default recomendado; puedes cambiarlo)
EMBEDDINGS_MODEL = os.getenv('VECTOR_EMBEDDINGS_MODEL', 'mixedbread-ai/mxbai-embed-large-v1')
NORMALIZE_EMBEDDINGS = True

# Performance knobs (útil para datasets grandes)
BATCH_SIZE = int(os.getenv('VECTOR_EMBEDDINGS_BATCH_SIZE', '64'))

# Device selection: usa GPU si torch + CUDA están disponibles
DEVICE = 'cpu'
try:
    import torch

    if torch.cuda.is_available():
        DEVICE = 'cuda'
except Exception:
    DEVICE = 'cpu'

# Si existe el artefacto, bórralo y reconstruye
REBUILD = True

print('REPO_ROOT:', REPO_ROOT)
print('DATASET_PATH:', DATASET_PATH)
print('VECTOR_DB_DIR:', VECTOR_DB_DIR)
print('MANIFEST_PATH:', MANIFEST_PATH)
print('ZIP_PATH:', ZIP_PATH)
print('VECTOR_COLLECTION:', VECTOR_COLLECTION)
print('EMBEDDINGS_MODEL:', EMBEDDINGS_MODEL)
print('DEVICE:', DEVICE)
print('BATCH_SIZE:', BATCH_SIZE)


## 4) Cargar dataset y construir documentos

Este notebook soporta 2 formatos de dataset:

1) **Fixture Django** (default en este repo): `backend/apps/libros/fixtures/libros_prueba.json`

2) **JSON genérico**: una lista de objetos con al menos `text` (o `page_content`) y opcional `id` y `metadata`.

Ejemplo JSON genérico:
```json
[
  {"id": "doc:1", "text": "Título: ...\nDescripción: ...", "metadata": {"source": "catalog"}},
  {"id": "doc:2", "text": "...", "metadata": {"category": "Ficción"}}
]
```

Para usar tu propio dataset:
- Exporta un JSON en uno de esos formatos.
- Define `VECTOR_DATASET_PATH` (env var) o edita `DATASET_PATH` en la sección de configuración.


In [ ]:
from langchain_core.documents import Document

if not DATASET_PATH.exists():
    raise FileNotFoundError(
        f'No se encontró DATASET_PATH={DATASET_PATH}. '
        'Ajusta VECTOR_DATASET_PATH (env var) o cambia DATASET_PATH en la sección de configuración.'
    )

raw = json.loads(DATASET_PATH.read_text(encoding='utf-8'))

# Soporta 2 formatos:
# A) Fixture Django (lista de objetos con model/pk/fields)
# B) JSON genérico (lista de dicts con al menos "text" y opcional "id"/"metadata")

def is_django_fixture(items) -> bool:
    if not isinstance(items, list) or not items:
        return False
    first = items[0]
    return isinstance(first, dict) and 'model' in first and 'pk' in first and 'fields' in first


docs: list[Document] = []
ids: list[str] = []
dataset_format = 'unknown'
counts: dict[str, int] = {}

if is_django_fixture(raw):
    dataset_format = 'django_fixture'

    categorias: dict[int, dict[str, Any]] = {}
    libros: list[dict[str, Any]] = []

    for item in raw:
        model = item.get('model')
        pk = item.get('pk')
        fields = item.get('fields', {})
        if model == 'libros.categoria':
            categorias[int(pk)] = fields
        elif model == 'libros.libro':
            libros.append({'pk': int(pk), **fields})

    def libro_to_text(libro: dict[str, Any]) -> str:
        categoria_nombre = categorias.get(int(libro.get('categoria') or 0), {}).get('nombre', '')
        parts = [
            f"Título: {libro.get('titulo','')}",
            f"Autor: {libro.get('autor','')}",
            f"Categoría: {categoria_nombre}" if categoria_nombre else None,
            f"Editorial: {libro.get('editorial','')}" if libro.get('editorial') else None,
            f"Año: {libro.get('año_publicacion','')}" if libro.get('año_publicacion') else None,
            f"ISBN: {libro.get('isbn','')}" if libro.get('isbn') else None,
            f"Descripción: {libro.get('descripcion','')}" if libro.get('descripcion') else None,
            f"Precio: {libro.get('precio','')}" if libro.get('precio') else None,
        ]
        return '\n'.join([p for p in parts if p])

    for libro in libros:
        categoria_id = int(libro.get('categoria') or 0)
        categoria_nombre = categorias.get(categoria_id, {}).get('nombre')
        metadata = {
            'libro_id': libro.get('pk'),
            'titulo': libro.get('titulo'),
            'autor': libro.get('autor'),
            'isbn': libro.get('isbn'),
            'categoria_id': categoria_id or None,
            'categoria': categoria_nombre,
            'editorial': libro.get('editorial'),
            'anio_publicacion': libro.get('año_publicacion'),
            'precio': libro.get('precio'),
            'stock': libro.get('stock'),
        }
        docs.append(Document(page_content=libro_to_text(libro), metadata=metadata))
        ids.append(f"libro:{libro.get('pk')}")

    counts = {
        'categorias': len(categorias),
        'libros': len(libros),
        'documents_indexed': len(docs),
    }

else:
    dataset_format = 'generic_json'

    if not isinstance(raw, list):
        raise ValueError('Formato no soportado: se esperaba una lista JSON.')

    for i, item in enumerate(raw):
        if not isinstance(item, dict):
            continue

        text = item.get('text') or item.get('page_content')
        if not text:
            continue

        metadata = item.get('metadata') or {}
        if not isinstance(metadata, dict):
            metadata = {'metadata': str(metadata)}

        doc_id = item.get('id') or item.get('doc_id') or f'doc:{i}'
        docs.append(Document(page_content=str(text), metadata=metadata))
        ids.append(str(doc_id))

    counts = {
        'records': len(raw),
        'documents_indexed': len(docs),
    }

print('dataset_format:', dataset_format)
print('docs:', len(docs))
print('sample doc:', (docs[0].page_content[:200] if docs else '<none>'))


## 5) Construir Chroma (persistente en disco)

Esto creará/actualizará la colección en `VECTOR_DB_DIR`.

Para datasets grandes, lo normal es correr esto en Colab (GPU) y luego bajar `vector_db.zip`.

In [ ]:
# Install dependencies for vector DB and embeddings
# Colab Scenario
# Temporarily install missing package if not in requirements.txt (ideally add to requirements.txt)
!pip install langchain-chroma
!pip install langchain-huggingface

In [ ]:
import shutil

from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

if REBUILD and VECTOR_DB_DIR.exists():
    shutil.rmtree(VECTOR_DB_DIR)

VECTOR_DB_DIR.mkdir(parents=True, exist_ok=True)

embeddings = HuggingFaceEmbeddings(
    model_name=EMBEDDINGS_MODEL,
    model_kwargs={'device': DEVICE},
    encode_kwargs={
        'normalize_embeddings': NORMALIZE_EMBEDDINGS,
        'batch_size': BATCH_SIZE,
    },
)

store = Chroma(
    collection_name=VECTOR_COLLECTION,
    embedding_function=embeddings,
    persist_directory=str(VECTOR_DB_DIR),
)

# Indexar (IDs estables por libro)
store.add_documents(documents=docs, ids=ids)

print('Indexado OK. Persist dir:', VECTOR_DB_DIR)
print('Colección:', VECTOR_COLLECTION)

## 6) Validación rápida (smoke test)

Ejecuta una búsqueda simple contra la DB para confirmar que el índice funciona y devuelve resultados plausibles.

In [ ]:
query = 'Cien años de soledad'
hits = store.similarity_search(query, k=3)

print('query:', query)
for i, d in enumerate(hits, start=1):
    md = d.metadata
    print(f"#{i} libro_id={md.get('libro_id')} | titulo={md.get('titulo')} | autor={md.get('autor')} | categoria={md.get('categoria')}")

## 7) Escribir `manifest.json`

El manifest te ayuda a asegurar reproducibilidad:
- qué embeddings/modelo se usó
- cuándo se construyó
- con qué dataset (hash)

Esto es clave cuando simulas un catálogo grande y haces rebuilds en Colab.

In [ ]:
def sha256_file(path: Path) -> str:
    h = hashlib.sha256()
    with path.open('rb') as f:
        for chunk in iter(lambda: f.read(1024 * 1024), b''):
            h.update(chunk)
    return h.hexdigest()


dataset_sha256 = sha256_file(DATASET_PATH)
built_at = datetime.now(timezone.utc).isoformat()

manifest = {
    'built_at_utc': built_at,
    'dataset': {
        'path': str(DATASET_PATH),
        'sha256': dataset_sha256,
        'format': dataset_format,
    },
    'vector_db': {
        'provider': 'chroma',
        'persist_directory': str(VECTOR_DB_DIR),
        'collection': VECTOR_COLLECTION,
    },
    'embeddings': {
        'model_name': EMBEDDINGS_MODEL,
        'normalize_embeddings': bool(NORMALIZE_EMBEDDINGS),
        'device': DEVICE,
        'batch_size': BATCH_SIZE,
    },
    'document_schema': {
        'notes': 'page_content es el texto embeddeado; metadata son campos de salida/filtros.',
    },
    'counts': counts,
}

MANIFEST_PATH.write_text(json.dumps(manifest, ensure_ascii=False, indent=2), encoding='utf-8')
print('Manifest escrito en:', MANIFEST_PATH)


## 8) Exportar `vector_db/` como ZIP (para llevarlo a otra máquina)

Esto es lo más útil cuando corres el build en Colab: te llevas el artefacto ya embeddeado a tu máquina local.

- Output: `backend/agent/vector_db.zip`

In [ ]:
import shutil

# Asegura que exista el artefacto
if not VECTOR_DB_DIR.exists():
    raise FileNotFoundError(f'No existe VECTOR_DB_DIR: {VECTOR_DB_DIR} (¿corriste el build?)')

# No generamos el manifest aquí: solo lo exportamos.
# Si no existe, falla para que corras el paso de manifest primero.
if not MANIFEST_PATH.exists():
    raise FileNotFoundError(
        f'No existe MANIFEST_PATH: {MANIFEST_PATH}. '
        'Corre primero el paso "Escribir manifest.json" y luego vuelve a exportar.'
    )

# Zippea la carpeta vector_db/ completa, preservando el directorio (vector_db/...) dentro del zip.
# Esto evita que al descomprimir quede todo "suelto" en la raíz.
if ZIP_PATH.exists():
    ZIP_PATH.unlink()

archive_base = str(ZIP_PATH.with_suffix(''))  # shutil.make_archive agrega .zip
shutil.make_archive(
    archive_base,
    'zip',
    root_dir=str(VECTOR_DB_DIR.parent),
    base_dir=str(VECTOR_DB_DIR.name),
)

print('ZIP creado:', ZIP_PATH)
print('Incluye manifest:', MANIFEST_PATH.name)
print('Tamaño (bytes):', ZIP_PATH.stat().st_size)
